In [1]:
import ufl

import dolfinx
from dolfinx import mesh, fem, io
from dolfinx.fem import FunctionSpace, VectorFunctionSpace, locate_dofs_geometrical, form
from dolfinx.fem.petsc import LinearProblem

from mpi4py import MPI
import petsc4py.PETSc
import numpy as np
import matplotlib.pyplot as plt
import shutil


# Operators:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)


dot = Infix(ufl.dot)
inner = Infix(ufl.inner)


def vector(*args):
    return ufl.as_vector(tuple(args))

from ufl import  TrialFunction, TestFunction, FacetNormal, lhs, rhs,SpatialCoordinate
from ufl import dx, ds, exp, sym, tr, sqrt
from ufl import nabla_div as div
from ufl import nabla_grad as grad
from ufl import as_matrix as matrix

npor = Infix(np.logical_or)
npand = Infix(np.logical_and)


def I(func_like):
    """Create matrix Identity dimension of func_like

    Args:
        func_like (Function): Give geometric dimension

    Returns:
        Tensor: Identity
    """
    return ufl.Identity(func_like.geometric_dimension())


# Classes:


def Function(space, func=None):
    """Function on new space. Without func just create a Function on space. 
    Args:
        space (FunctionSpace): New space
        function (): 
    \nScalars - fem.Function,fem.Constant, ufl_function, callable function, number
    \nVectors - fem.vector_Function,fem.vector_Constant, ufl_vector_function, callable vector_function, tuple_number
    Returns:
        fem.Function: Function
    """

    result = fem.Function(space)
    tupe = str(func.__class__)[8:-2]
    cord = SpatialCoordinate(space)

    # None
    if func == None:
        return result

    # fem.Function
    elif tupe == ('dolfinx.fem.function.Function'):
        expression = func

    # fem.Constant
    elif tupe == ('dolfinx.fem.function.Constant'):
        if len(func.ufl_shape) == 0: func2 = func.value + (cord[0] - cord[0])
        else:
            func2 = vector(*func.value) +\
                vector(*map(lambda x, y: x - y, cord, cord))
        expression = fem.Expression(func2, space.element.interpolation_points())

    # ufl object
    elif tupe[:3] == 'ufl':
        if len(func.ufl_shape) != 0:
            func2 = func + vector(*map(lambda x, y: x - y, cord, cord))
        else:
            func2 = func
        expression = fem.Expression(func2, space.element.interpolation_points())

    # Python function
    elif tupe == ('function'):
        expression = func

        # Number
    elif not hasattr(func, '__call__'):
        if hasattr(func, '__getitem__'):
            func2 = vector(*func) + vector(*map(lambda x, y: x - y, cord, cord))
        else:
            func2 = func + (cord[0] - cord[0])
        expression = fem.Expression(func2, space.element.interpolation_points())

    result.interpolate(expression)
    return result


def Constant(domain_space, const):
    """Constant on space

    Args:
        space (fem.FunctionSpace| domain): Space or domain
        const (auny number): Any number

    Returns:
        fem.function.Constant: Constant on space
    """
    return fem.Constant(domain_space, petsc4py.PETSc.ScalarType(const))


def create_facet_markers(domain, bound_markers):
    """Mark facets under conditious

    Args:
        domain (Domain): Domain
        bound_markers (mark,python_function): List of mark and function
    
    Return:
        tags(mesh.meshtags): Marked facets
    """
    facet_indices, facet_markers = [], []
    for (marker, condition) in bound_markers:
        facets = mesh.locate_entities(
            domain,
            domain.topology.dim - 1,
            condition,
            )
        facet_indices.append(facets)
        facet_markers.append(np.full_like(facets, marker))
    facet_indices = np.hstack(facet_indices).astype(np.int32)
    facet_markers = np.hstack(facet_markers).astype(np.int32)
    sorted_facets = np.argsort(facet_indices)
    facet_tags = mesh.meshtags(
        domain,
        domain.topology.dim - 1,
        facet_indices[sorted_facets],
        facet_markers[sorted_facets],
        )

    return facet_tags


def create_connectivity(domain):
    """Need to compute facets to Boundary value

    Args:
        domain (Mesh): Domain
    """
    domain.topology.create_connectivity(
        domain.topology.dim - 1,
        domain.topology.dim,
        )


def DirichletBC(space, func, combined_marker):
    """Create Dirichlet condition

    Args:
        space (fem.FunctionSpace): Function space
        func (fem.function): Function or Constant
        combined_marker (Any): One from next
        \nFunction - boundary marker function
        \nAll - all boundary
        \n(mesh.meshtags, marker) - list or tuple, marker of boundary from Marked_facets - mesh.meshtags
        
    Returns:
        condition (dirichletbc): Dirichlet condition
    """

    def get_exterior_facets():
        exterior_facets = mesh.exterior_facet_indices(domain.topology)
        return exterior_facets

    def all_dirichlet(dofs, func):
        if hasattr(func, 'function_space'):
            bc = fem.dirichletbc(dofs=dofs, value=func)
        else:
            bc = fem.dirichletbc(V=space, dofs=dofs, value=func)
        return bc

    domain = space.mesh

    if combined_marker == 'All':
        facets = get_exterior_facets()
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )

    elif type(combined_marker) == (tuple or list):
        marked_facets, marker = combined_marker
        facets = marked_facets.find(marker)
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )
    else:
        dofs = fem.locate_dofs_geometrical(space, marker=combined_marker)

    bc = all_dirichlet(dofs, func)

    return bc


# Post processing:

# Fix it
def errors_L(space , uS, uEx):
    """Compute error norm on boundary

    Args:
        uS (Function): Numeric solution
        uEx (Function): Exact or model solution

    Returns:
        List: L1 and L2 norms
    """
    domain = space.mesh

    L1_scalar = fem.assemble_scalar(form((uS-uEx) * dx))
    L2_scalar = fem.assemble_scalar(form((uS - uEx)**2 * dx))

    L1_err = np.abs(domain.comm.allreduce(L1_scalar, op=MPI.SUM))
    L2_err = np.sqrt(domain.comm.allreduce(L2_scalar, op=MPI.SUM))
    return (L1_err, L2_err)


def line_collision(domain, line_cord):
    """Generate points and cells of colliding domain and line

    Args:
        domain (mesh): Domain
        line_cord (array): 3D line contervertor of coordinates 

    Returns:
        Tuple: Collision points and collision cells
    """
    bb_tree = dolfinx.geometry.BoundingBoxTree(domain, domain.topology.dim)

    cells_on_line = []
    points_on_line = []
    cell_candidates = dolfinx.geometry.compute_collisions(bb_tree, line_cord.T)
    colliding_cells = dolfinx.geometry.compute_colliding_cells(
        domain, cell_candidates, line_cord.T
        )
    for i, point in enumerate(line_cord.T):
        if len(colliding_cells.links(i)) > 0:
            points_on_line.append(point)
            cells_on_line.append(colliding_cells.links(i)[0])

    points_on_line = np.array(points_on_line, dtype=np.float64)

    return (points_on_line, cells_on_line)


def graph2D(fig, lists, natural_show=False, points_on=False):
    """Create graph from fem.Function

    Args:
        fig (plt.Figure): Figure
        lists (fem.Function , plt.Axes, str): List of (u, curent axes, title)
        method (bool): Graph method True = tripcolor, False = tricontourf
    """

    def data_construct(dofs, x_array):
        data = np.column_stack((dofs[:, 0:2], x_array))
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        return [x_data, y_data, z_data]

    for list in lists:
        fig, ax = plt.subplots()
        plt.close()
        u, ax, title = list
        dofs = u.function_space.tabulate_dof_coordinates()
        ax.set_title(title)
        data = data_construct(dofs, u.x.array)

        if points_on: ax.plot(data[0], data[1], 'o', markersize=2, color='grey')

        if natural_show:
            plot = ax.tripcolor(*data)
        else:
            try:
                levels = np.linspace(u.x.array.min(), u.x.array.max(), 10)
                plot = ax.tricontourf(
                    *data,
                    levels=levels,
                    )
            except:
                print(f'{title} - error')

        ax.set_aspect(1)
        fig.colorbar(plot, ax=ax)
    return

In [2]:
N = 50
T = 1
Nt = 100

dt = T / Nt
domain = mesh.create_unit_square(nx=N, ny=N, comm=MPI.COMM_WORLD)
create_connectivity(domain=domain)

# v_element = ufl.VectorElement('CG',domain.ufl_cell(),2)
# s_element = ufl.FiniteElement('CG',domain.ufl_cell(),1)
# V = FunctionSpace(domain, v_element)
# Q =FunctionSpace(domain, s_element)

V = VectorFunctionSpace(domain, ('CG', 2))
Q = FunctionSpace(domain, ('CG', 2))
x, y = SpatialCoordinate(V)
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure('cell', subdomain_id='everywhere')
n = FacetNormal(domain)

u, v = TrialFunction(V), TestFunction(V)
p, q = TrialFunction(Q), TestFunction(Q)

rho = Constant(domain, 1)
mu = Constant(domain, 1)
f = Constant(V, (0, 0))

uDV = vector(0, 0)
uDQ1 = 8
uDQ2 = 0

sigma = lambda u, p: 2 * mu * sym(grad(u)) - p * I(u)

bound_markers = [
    (1, lambda x: np.isclose(x[0], 0)),
    (2, lambda x: np.isclose(x[0], 1)),
    (3, lambda x: np.isclose(x[1], 0)),
    (4, lambda x: np.isclose(x[1], 1)),
    ]

marked_facets = create_facet_markers(
    domain=domain,
    bound_markers=bound_markers,
    )

bcsV = [
    DirichletBC(V, Function(V, uDV), (marked_facets, 3)),
    DirichletBC(V, Function(V, uDV), (marked_facets, 4))
    ]

bcsQ = [
    DirichletBC(Q, Function(Q, uDQ1), (marked_facets, 1)),
    DirichletBC(Q, Function(Q, uDQ2), (marked_facets, 2))
    ]

u0 = Function(V)
u0.name = 'Velocity'
uS = Function(V)
U = 0.5 * (u0+u)
p0 = Function(Q)
p0.name = 'Pressure'
pS = Function(Q)

F1 = (1/dt) * rho * ((u - u0)|dot|v) * dx + rho * ((u0|dot|grad(u0))|dot|v) * dx
F1 += (sigma(U, p0)|inner|sym(grad(v))) * dx
F1 += (p0 * n|dot|v) * ds - mu * ((grad(U) * n)|dot|v) * ds
F1 += -(f|dot|v) * dx

F2 = (grad(p)|dot|grad(q)) * dx - (grad(p0)|dot|grad(q)) * dx
F2 += (1/dt) * (div(uS)|dot|q) * dx

F3 = ((u - uS)|dot|v) * dx + dt * (grad(pS - p0)|dot|v) * dx


In [3]:
a1 = form(lhs(F1))
L1 = form(rhs(F1))
A1 = fem.petsc.assemble_matrix(a1,bcsV)
A1.assemble()
b1 = fem.petsc.create_vector(L1)

a2 = form(lhs(F2))
L2 = form(rhs(F2))
A2 = fem.petsc.assemble_matrix(a2,bcsQ)
A2.assemble()
b2 = fem.petsc.create_vector(L2)


a3 = form(lhs(F3))
L3 = form(rhs(F3))
A3 = fem.petsc.assemble_matrix(a3,)
A3.assemble()
b3 = fem.petsc.create_vector(L3)

In [4]:
solver1 = petsc4py.PETSc.KSP().create(domain.comm)
solver1.setOperators(A1)
solver1.setType(petsc4py.PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(petsc4py.PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = petsc4py.PETSc.KSP().create(domain.comm)
solver2.setOperators(A2)
solver2.setType(petsc4py.PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(petsc4py.PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = petsc4py.PETSc.KSP().create(domain.comm)
solver3.setOperators(A3)
solver3.setType(petsc4py.PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(petsc4py.PETSc.PC.Type.SOR)

In [5]:
u_start = Function(V,u0)
p_start = Function(Q,p0)
uEx = Function(V, (4*y*(1-y),0))

In [6]:
try:
    shutil.rmtree('/home/VTK/Stokes_Files')
except:
    print('Directory empty yet')

Directory empty yet


In [7]:
with io.VTKFile(domain.comm, '/home/VTK/Stokes_Files/stokes.pvd', 'w') as file:
    for t in np.arange(0, T + dt, dt):
        u0.interpolate(uS)
        p0.interpolate(pS)
        file.write_function([u0, p0], t)

        # Step 1
        # Cleaning linear_form solution
        with b1.localForm() as loc_b1:
            loc_b1.set(0)
        fem.petsc.assemble_vector(b1, L1)
        # Apply Dirichlet to the b
        fem.petsc.apply_lifting(b1, [a1], [bcsV])
        b1.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.ADD_VALUES,
            mode=petsc4py.PETSc.ScatterMode.REVERSE,
            )
        fem.petsc.set_bc(b1, bcsV)
        # Solving
        solver1.solve(b1, uS.vector)
        uS.x.scatter_forward()

        # Step 2
        # Cleaning linear_form solution
        with b2.localForm() as loc_b2:
            loc_b2.set(0)
        fem.petsc.assemble_vector(b2, L2)
        # Apply Dirichlet to the b
        fem.petsc.apply_lifting(b2, [a2], [bcsQ])
        b2.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.ADD_VALUES,
            mode=petsc4py.PETSc.ScatterMode.REVERSE,
            )
        fem.petsc.set_bc(b2, bcsQ)
        # Solving
        solver2.solve(b2, pS.vector)
        pS.x.scatter_forward()

        # Step 3
        # Cleaning linear_form solution
        with b3.localForm() as loc_b3:
            loc_b3.set(0)
        fem.petsc.assemble_vector(b3, L3)
        b3.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.ADD_VALUES,
            mode=petsc4py.PETSc.ScatterMode.REVERSE,
            )
        # Solving
        solver3.solve(b3, uS.vector)
        uS.x.scatter_forward()

In [8]:
# fig, (ax, ax2) = plt.subplots(1, 2)
# fig.set_size_inches(16, 8)
# graph2D(
#     fig=fig,
#     lists=[
#         [u1, ax, 'u start'],
#         [u0, ax2, 'u end'],
#         ],
#     natural_show=True,   
#     # points_on=True,
#     )